<a href="https://colab.research.google.com/github/yueguo1997/SNA_recommender_system/blob/main/SNA_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data preparation

In [2]:
cur_path = "/content/drive/My Drive/SNA final project/"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/SNA final project


In [3]:
# Reading in datasets/book1.csv
df = pd.read_csv('bonanza_buyer_item_seller_id_R_2.txt', sep='::', header=None, names=['buyer', 'item', 'seller', 'timestamp'])

# Printing out the head of the dataset
# split the data into train and test
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

<ipython-input-3-8e4ad435eb4a>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('bonanza_buyer_item_seller_id_R_2.txt', sep='::', header=None, names=['buyer', 'item', 'seller', 'timestamp'])


# Create Graph

In [5]:
import networkx as nx
import csv

In [9]:
# give each node unique node id
nodes_type = ['buyer', 'item', 'seller']
index = 0

dictionary = {}
for t in nodes_type:
  for i in range(len(df)):
    if (df[t][i],t) not in dictionary:
      dictionary[(df[t][i],t)] = index
      index += 1




In [ ]:
G_train = nx.MultiGraph()
G_test = nx.MultiGraph()

for i in range(len(train_data)):
  

In [159]:


# Add nodes for buyers, items, and sellers
for node_type in ['buyer', 'item', 'seller']:
    nodes1 = train_data[node_type].unique()
    nodes2 = test_data[node_type].unique()
    for node in nodes1:
      
        G_train.add_node(node, type=node_type)
    for node in nodes2:
      if node == 482:
        print(node_type)
        G_test.add_node(node, type=node_type)

# Add edges for interactions
for _, row in train_data.iterrows():
    buyer = row['buyer']
    item = row['item']
    seller = row['seller']
    time = row['timestamp']
    G_train.add_edge(buyer, item, type='buys', timestamp = time)
    G_train.add_edge(item, seller, type='sold_by')
for _, row in test_data.iterrows():
    buyer = row['buyer']
    item = row['item']
    seller = row['seller']
    time = row['timestamp']
    G_test.add_edge(buyer, item, type='buys', timestamp = time)
    G_test.add_edge(item, seller, type='sold_by')

buyer
item
seller


In [153]:
# Print some statistics about the graph
print("Number of nodes on train:", G_train.number_of_nodes())
print("Number of edges on train:", G_train.number_of_edges())
print("Number of unique edge types on train:", len(set(nx.get_edge_attributes(G_train, 'type').values())))

print("Number of nodes on test:", G_test.number_of_nodes())
print("Number of edges on test:", G_test.number_of_edges())
print("Number of unique edge types on test:", len(set(nx.get_edge_attributes(G_test, 'type').values())))

Number of nodes on train: 8501
Number of edges on train: 62560
Number of unique edge types on train: 2
Number of nodes on test: 5297
Number of edges on test: 15640
Number of unique edge types on test: 2


# Node embedding vased on deepwalk

# Model training

In [164]:
# Define the GNN model
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GATConv(in_channels=16, out_channels=32, heads=4)
        self.conv2 = GATConv(in_channels=32, out_channels=1, heads=1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x)



In [165]:
# Define the training loop
def train(model, optimizer, x, y, edge_index):
    model.train()
    optimizer.zero_grad()
    out = model(x, edge_index)
    loss = F.binary_cross_entropy(out, y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Define the test loop
def test(model, x, y, edge_index):
    model.eval()
    with torch.no_grad():
        out = model(x, edge_index)
        pred = (out > 0.5).float()
        acc = (pred == y).sum().item() / y.size(0)
    return acc



In [ ]:
# Prepare the data
x = torch.randn(len(buyer) + len(seller) + len(item), 16)
y = torch.cat([torch.ones(len(buyer)), torch.zeros(len(seller) + len(item))])
data = Data(x=x, edge_index=edge_index, y=y)

# Initialize the model and optimizer
model = GNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
for epoch in range(200):
    loss = train(model, optimizer, data.x, data.y, data.edge_index)
    acc = test(model, data.x, data.y, data.edge_index)
    print(f'Epoch: {epoch + 1}, Loss: {loss:.4f}, Accuracy: {acc:.4f}')

In [167]:
G_train.nodes[0]

{'type': 'seller'}